In [ ]:
import os
import numpy as np
import pandas as pd
os.chdir('/content')
!mkdir /root/.kaggle
!echo '{"username":"daniel1998xx","key":"c99226b7cd73b12d4681a78a5b02d227"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts
if not os.path.exists("/content/NPR_Data"):
    os.makedirs("/content/NPR_Data")


 97% 285M/295M [00:02<00:00, 150MB/s]
100% 295M/295M [00:02<00:00, 137MB/s]


In [ ]:
!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data

Archive:  interview-npr-media-dialog-transcripts.zip
  inflating: NPR_Data/episodes.csv   
  inflating: NPR_Data/headlines.csv  
  inflating: NPR_Data/host-map.json  
  inflating: NPR_Data/host_id.json   
  inflating: NPR_Data/splits-ns2.json  
  inflating: NPR_Data/utterances-2sp.csv  
  inflating: NPR_Data/utterances.csv  


In [ ]:
os.chdir('/content/NPR_Data')

In [ ]:
df = pd.read_csv('utterances.csv')
df['speaker'] = df['speaker'].str.upper()
df['speaker'] = df['speaker'].str.split(', H', expand=True)[0]

In [ ]:
df = df[df['speaker'] != '_NO_SPEAKER']

In [ ]:
df['utterance_len'] = df['utterance'].str.len()

In [ ]:
df = df.dropna()
host_id_df = pd.read_json('host_id.json', orient='index')

In [ ]:
host_id_df = pd.read_json('host_id.json', orient='index')
host_id_df.reset_index(inplace=True)
host_id_df = host_id_df.rename(columns = {'index':'speaker'})

In [ ]:
host_id_df['speaker'] = host_id_df['speaker'].astype('string')
host_id_df['speaker'] = host_id_df['speaker'].str.upper()
host_id_df.rename(columns={0:'host_id'}, inplace=True)

In [ ]:
mergedf = df.merge(host_id_df, how='left', on='speaker')

In [ ]:
mergedf['host_id'] = mergedf['host_id'].fillna(-1)

In [ ]:
PreProcess_df = mergedf[mergedf['host_id'] != -1]

In [ ]:
PreProcess_df = PreProcess_df.loc[PreProcess_df['utterance_len'] >= 100]

In [ ]:
top_speakers = PreProcess_df.groupby(['speaker']).size().loc[PreProcess_df.groupby(['speaker']).size() > 30000]
df = pd.DataFrame(PreProcess_df.loc[PreProcess_df['speaker'].isin(top_speakers.index.values)])
df = df.reset_index(drop=True)

In [ ]:
df['utterance'] = df['utterance'].str.lower()

In [ ]:
df['unidentified'] = df['utterance'].str.startswith('unidentified')
df = df[df['unidentified'] == False]

In [ ]:
df.count()

episode          220594
episode_order    220594
speaker          220594
utterance        220594
utterance_len    220594
host_id          220594
unidentified     220594
dtype: int64

In [ ]:
df['speaker'].value_counts()

NEAL CONAN       126971
STEVE INSKEEP     33787
ROBERT SIEGEL     29948
IRA FLATOW        29888
Name: speaker, dtype: int64

In [ ]:
df_prac = df.groupby('speaker').apply(lambda x: x.sample(34000) if x['speaker'].iloc[0] == 'NEAL CONAN' else x)
df_prac = df_prac.reset_index(drop = True)

In [ ]:
df_prac['speaker'].value_counts()

NEAL CONAN       34000
STEVE INSKEEP    33787
ROBERT SIEGEL    29948
IRA FLATOW       29888
Name: speaker, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def preprocess_text(text):
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]

    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

In [ ]:
utterance_list = df_prac['utterance'].to_list()
final_list = []
complete_list = []
for utterance in utterance_list:
    processed = preprocess_text(utterance)
    final_list.append(processed)
complete_list.extend(final_list)
df_prac['processed_utterance'] = complete_list

In [ ]:
df_prac['processed_utterance'] = df_prac['processed_utterance'].apply(lambda x: ' '.join(x))

In [ ]:
df_prac


,episode,episode_order,speaker,utterance,utterance_len,host_id,unidentified,processed_utterance
0,68175,0,IRA FLATOW,this is talk of the nation: science friday. i...,377.0,8.0,False,talk nation science friday ira flatow tucson a...
1,68175,1,IRA FLATOW,but something i learned on this trip out west ...,293.0,8.0,False,something learn trip west chilies make food ho...
2,68175,2,IRA FLATOW,that stuff that makes them hot is also used fo...,439.0,8.0,False,stuff make hot also use pain reliever see shak...
3,68175,3,IRA FLATOW,why do we like something that's that hot for u...,366.0,8.0,False,like something hot us burn maybe burn tongue f...
4,68175,4,IRA FLATOW,"the appeal to ask a question? also, our numbe...",393.0,8.0,False,appeal ask question also number 1 800 989 8255...
...,...,...,...,...,...,...,...,...
127618,132036,4,STEVE INSKEEP,"robin moore of hoquiam, washington, felt jarre...",467.0,16.0,False,robin moore hoquiam washington felt jar mr sie...
127619,132036,6,STEVE INSKEEP,some responses to our story on internet access...,427.0,16.0,False,responses story internet access power line say...
127620,132036,8,STEVE INSKEEP,"but christina tracy of lookout mountain, georg...",538.0,16.0,False,christina tracy lookout mountain georgia write...
127621,132036,12,STEVE INSKEEP,we don't necessarily need to hear about your t...,169.0,16.0,False,necessarily need hear tan line welcome drop us...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import make_pipeline

In [ ]:
X = df_prac.processed_utterance
y = df_prac.speaker
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.steps

param_grid = {}
param_grid["tfidfvectorizer__max_features"] = [500, 1000, 15000]
param_grid["tfidfvectorizer__ngram_range"] = [(1,1), (1,2), (2,2)]
#param_grid["tfidfvectorizer__stop_words"] = ["english", None]
param_grid["tfidfvectorizer__strip_accents"] = ["ascii", "unicode", None]
param_grid["tfidfvectorizer__analyzer"] = ["word", "char"]
param_grid["tfidfvectorizer__binary"] = [True, False]
param_grid["tfidfvectorizer__norm"] = ["l1", "l2", None]
param_grid["tfidfvectorizer__use_idf"] = [True, False]
param_grid["tfidfvectorizer__smooth_idf"] = [True, False]
param_grid["tfidfvectorizer__sublinear_tf"] = [True, False]


grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', verbose = 3, n_jobs = -1)
grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__tfidfvectorizer', 'estimator__multinomialnb', 'estimator__tfidfvectorizer__analyzer', 'estimator__tfidfvectorizer__binary', 'estimator__tfidfvectorizer__decode_error', 'estimator__tfidfvectorizer__dtype', 'estimator__tfidfvectorizer__encoding', 'estimator__tfidfvectorizer__input', 'estimator__tfidfvectorizer__lowercase', 'estimator__tfidfvectorizer__max_df', 'estimator__tfidfvectorizer__max_features', 'estimator__tfidfvectorizer__min_df', 'estimator__tfidfvectorizer__ngram_range', 'estimator__tfidfvectorizer__norm', 'estimator__tfidfvectorizer__preprocessor', 'estimator__tfidfvectorizer__smooth_idf', 'estimator__tfidfvectorizer__stop_words', 'estimator__tfidfvectorizer__strip_accents', 'estimator__tfidfvectorizer__sublinear_tf', 'estimator__tfidfvectorizer__token_pattern', 'estimator__tfidfvectorizer__tokenizer', 'estimator__tfidfvectorizer__use_idf', 'estimator__tfidfvectorizer__

In [ ]:
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits


KeyboardInterrupt: ignored

In [ ]:
vector = TfidfVectorizer(analyzer='word', max_features = 850, ngram_range=(1, 1), 
                       binary=False, norm=None, smooth_idf=True, strip_accents=None,
                       sublinear_tf=True, use_idf=False)

df_transformed = vector.fit_transform(X_train)

In [ ]:
multi_nb_model = MultinomialNB()
multi_nb_model.fit(df_transformed, y_train)
print ("Model accuracy within dataset: ", multi_nb_model.score(df_transformed, y_train))

Model accuracy within dataset:  0.6100797141573597


In [ ]:
test_text = ["So for a top competitor like Lance to try to"]
test_text_transform = vector.transform(test_text)

print (multi_nb_model.predict(test_text_transform)," most likely said it.")

['STEVE INSKEEP']  most likely said it.


In [ ]:
test_text = ["something learn trip west chilies make food"]
test_text_transform = vector.transform(test_text)

print (multi_nb_model.predict(test_text_transform)," most likely said it.")

['IRA FLATOW']  most likely said it.
